In [1]:
# ! pip3 install selenium
# brew install geckodriver

  Using cached selenium-3.141.0-py2.py3-none-any.whl (904 kB)
  Using cached urllib3-1.26.6-py2.py3-none-any.whl (138 kB)
You should consider upgrading via the '/Users/umariqbal/Documents/voice-assistant-central/env/bin/python3.9 -m pip install --upgrade pip' command.


In [2]:
from selenium import webdriver
import time

In [22]:
def signin(singin_page):
    #     firefox_options = webdriver.FirefoxOptions()
    #     firefox_options.add_argument('--disable-application-cache')
    #     firefox_options.add_argument('--disable-infobars')
    #     firefox_options.add_argument('--no-sandbox')    
    #     firefox_options.add_argument('--chrome-binary=' + binary_path)
    #     firefox_options.binary_location = binary_path
    driver = webdriver.Firefox()
    
    try:
        driver.get(singin_page)
        time.sleep(3)

        signin_button = driver.find_element_by_id('a-autoid-0-announce')
        signin_button.click()
        time.sleep(3)
        
        email_input = driver.find_element_by_id('ap_email')
        email_input.send_keys('phillip-phillips-1@protonmail.com')
        time.sleep(3)
        
        continue_button = driver.find_element_by_id('continue')
        continue_button.click()
        time.sleep(3)
        
        
        password_input = driver.find_element_by_id('ap_password')
        password_input.send_keys('change.me')
        time.sleep(3)
        
        signin_button = driver.find_element_by_id('signInSubmit')
        signin_button.click()
        time.sleep(3)
        
        skill_enable_button = driver.find_element_by_id("a2s-skill-enable-button")
        skill_enable_button.click()
        time.sleep(3)
        
        permission_button = driver.find_element_by_id("a-button-input")
        permission_button.click()
        time.sleep(10)
        
    except BaseException as ex:
        print('Something went wrong: ', str(ex))
        pass

    finally:
        driver.quit()
    

def crawl(webpage_to_open):
#     firefox_options = webdriver.FirefoxOptions()
#     firefox_options.add_argument('--disable-application-cache')
#     firefox_options.add_argument('--disable-infobars')
#     firefox_options.add_argument('--no-sandbox')    
#     firefox_options.add_argument('--chrome-binary=' + binary_path)
#     firefox_options.binary_location = binary_path
    driver = webdriver.Firefox()
    
    try:
        driver.get(webpage_to_open)
        time.sleep(3)
        skill_enable_button = driver.find_element_by_id("a2s-skill-enable-button")
        
        skill_enable_button.click()
        time.sleep(10)
    except BaseException as ex:
        print('Something went wrong: ', str(ex))
        pass

    finally:
        driver.quit()

In [23]:
signin('https://www.amazon.com/Dominos-Pizza-LLC/dp/B01B5G99CC/ref=lp_14284822011_1_7?s=digital-skills&ie=UTF8')
# crawl('https://www.amazon.com/Dominos-Pizza-LLC/dp/B01B5G99CC/ref=lp_14284822011_1_7?s=digital-skills&ie=UTF8')

Something went wrong:  Message: Unable to locate element: [id="a-button-input"]

